In [1]:
#Importing the usual suspects
import numpy as np
import sys
import os
np.set_printoptions(threshold=sys.maxsize)

parent = os.getcwd()
os.chdir(parent+'//output_fold_1')
import warnings
warnings.simplefilter(action='ignore') #stops annoying pandas warning
from landlab import RasterModelGrid #sets up grid
from landlab.components import FlowAccumulator #finds drainage area
from landlab.components import FastscapeEroder #employs SPIM
from landlab.components import Lithology
from landlab.components import DepressionFinderAndRouter
from landlab.components import ChiFinder
from layer_functions import *

np.random.seed(12345)
multiplier = 0.2
rows = 50
columns = 100
dx = 100. #meters
dt = 100. #time step, yrs
T = 6. * 10. ** 6. #simulation time, yrs
uplift = 0.001 #uplift rate, m/yr

mg = RasterModelGrid((rows,columns),dx) #The Grid
eta = mg.add_zeros('topographic__elevation', at = 'node')
mg.at_node['topographic__elevation'][mg.core_nodes] = np.random.rand(len(mg.core_nodes)) #resetting topography
jeff_bc_open_or_closed(mg, False, False, True, False)

#D8 is the method of finding flow direction
flow = FlowAccumulator(mg, flow_director='D8')#,depression_finder=DepressionFinderAndRouter)
#K_sp is K in the SPIM equation
erode = FastscapeEroder(mg, K_sp=0.00001)
cf = ChiFinder(mg,min_drainage_area=0.0,reference_concavity=.5,heterogenous=True,uplift=uplift)#,use_true_dx=True)
'''
fault = mg.add_zeros('fault', at = 'node')
add_fault_lines(mg,'fault',-2.,mg.dx * 25.,False,False,T/6.*uplift)
add_fault_lines(mg,'fault',0.5,mg.dx * 25.,False,False,T/6.*uplift)
#add_fault_lines(mg,'fault',0.0,mg.dx * 20.,True,False,T/6.*uplift)
#add_fault_lines(mg,'fault',0.0,mg.dx * 20.,False,True,T/6.*uplift)
jeff_plot(mg,'fault',False,'inferno','fault','Fault.png',0)  
non_fault = np.zeros_like(fault)
non_fault[fault==0.0] = 10000.
thicknesses = [T/3.*uplift+non_fault,fault,100000]
'''                      
fold = mg.add_zeros('fold', at = 'node')
fold_thickness = T/6.*uplift
#add_folds(mg,'fold',0.0,mg.extent[0]/3.,T/6.*uplift,True,False)
#jeff_plot(mg,'fold',False,'inferno','fold','Fold_horiz.png',0)       
add_folds(mg,'fold',0.0,mg.extent[1]/2.0,T/6.*uplift,False,True)
jeff_plot(mg,'fold',False,'inferno','fold','Fold_vert.png',0)        
#add_folds(mg,'fold',2.0,mg.extent[1]/5.0,T/2.*uplift,False,False)
#jeff_plot(mg,'fold',False,'inferno','fold','Fold_sloped.png',0)     
thicknesses = [T/3.*uplift + fold,fold_thickness,100000]

ids = [1, 2, 1]
attrs = {'K_sp': {1: 0.00001, 2: 0.00001*multiplier}}
lith = Lithology(mg, thicknesses, ids, attrs, dz_advection = uplift * dt)
lith_elevation = np.zeros_like(lith.dz)
lith_rock_ksp = np.zeros(lith.dz.shape[0])
lith_elevation[0,:] = eta - lith.dz[-1,:]
for i in range(0,lith.dz.shape[0]):
    lith_rock_ksp[i] = attrs['K_sp'][ids[i]]
j = 1
for i in range(lith.dz.shape[0]-2,-1,-1):
    lith_elevation[j,:] = lith_elevation[j-1,:] - lith.dz[i,:]
    j+=1
lith.run_one_step()
flow.run_one_step() #find drainage area

for t in range(0,int(T/dt*2.0)+1):    
    if t%(int(T/dt)/20) == 0: #progress bar
    #if t >= T/dt/3 and  t <= T/dt/2 and t%100 == 0:
        #multirock_plot(mg,attrs['K_sp'][1],t) 
        cf.calculate_chi()     
        #multirock_profiler(mg,lith,uplift,t)
        jeff_plot(mg,'channel__chi_index',False,'viridis',r'$\chi$ [m]','Chi_'+ '%06d' % t + '.png',0)    
        jeff_plot(mg,'topographic__elevation',False,'viridis',r'$\eta$ [m]','Topography_'+ '%06d' % t + '.png',0)   
        jeff_plot(mg,'drainage_area',True,'inferno',r'log(A) [log m$^2$]','Drainage_Area_'+ '%06d' % t + '.png',0)
        jeff_plot(mg,'K_sp',False,'gray',r'K [$yr^{-1}$]','Erodibility_'+ '%06d' % t + '.png',1)
        multirock_profiler(mg,lith_elevation,lith_rock_ksp,0,t,uplift,dt)
        #jeff_text(mg,lith,'_'+ '%06d' % t)
        print (str(round(float(t)/float(T/dt)*100,1)) +'% ', end = '')
    erode = FastscapeEroder(mg, K_sp = mg.at_node['K_sp'], m_sp = 0.5, n_sp = 1.0)
    flow.run_one_step() #find drainage area
    erode.run_one_step(dt=dt) #calculate erosion rate and subtract from the topography
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift * dt #add uplift
    lith.run_one_step()

#jeff_plot(mg,'topographic__elevation',False,'viridis',r'$\eta$ [m]','Topography_'+ str(multiplier) + '.png')   
#jeff_plot(mg,'drainage_area',True,'inferno',r'log(A) [log m$^2$]','Drainage_Area_'+ str(multiplier) +'.png')
print ('done!') 


0.0% 5.0% 10.0% 15.0% 20.0% 25.0% 30.0% 35.0% 40.0% 45.0% 50.0% 55.0% 60.0% 65.0% 70.0% 75.0% 80.0% 85.0% 90.0% 95.0% 100.0% 105.0% 110.0% 115.0% 120.0% 125.0% 130.0% 135.0% 140.0% 145.0% 150.0% 155.0% 160.0% 165.0% 170.0% 175.0% 180.0% 185.0% 190.0% 195.0% 200.0% done!


In [2]:
attrs['K_sp']

{1: 1e-05, 2: 2.0000000000000003e-06}

In [3]:
print (mg.at_node.keys())

['topographic__elevation', 'water__unit_flux_in', 'drainage_area', 'flow__data_structure_delta', 'flow__upstream_node_order', 'surface_water__discharge', 'flow__sink_flag', 'flow__link_to_receiver_node', 'flow__receiver_node', 'topographic__steepest_slope', 'channel__chi_index', 'fold', 'K_sp', 'rock_type__id']
